In [ ]:
# https://www.umwelt.steiermark.at/cms/ziel/2060750/DE/
# https://www.data.gv.at/katalog/api/3/action/package_show?id=58a2e0c0-1cf9-11e2-892e-0800200c9a66
# https://www.data.gv.at/katalog/dataset/58a2e0c0-1cf9-11e2-892e-0800200c9a66

In [1]:
import requests
import time
import os

import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

s = Service(ChromeDriverManager().install())

link = 'http://app.luis.steiermark.at/luft2/suche.php?'



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/home/pseebacher/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache


In [2]:
mittelwert_vals = {
    'HMW (Halbstundenmittelwert)': 1,
    'MW1 (Gleit. Einstundenmittelwert)': 2,
    'MW3 (Gleit. Dreistundenmittelwert)': 4,
    'TMW (Tagesmittelwert)': 21
}

komponente = {
    'SO2': 1,
    'NO': 3,
    'NO2': 4,
    'NOX': 5,    
#     'Schwebstaub': 7,
    'Lufttemperatur (LUTE)': 8,
    'Relative Luftfeuchte (LUFE)': 9,
    'PM10 Kont. (STBK 10G)': 114,
#     'PM10 Grav. (STBK 10G)': 119,
#     'PM2,5 Grav. (STBK 25G)': 122,
    'Feinstabu (PM10)': 125
}

# stations = {
#     'Don Bosco': 164,
#     'Ost Petergasse': 171,
#     'Sud Tiergartenweg': 170,
#     'West': 139,
#     'Nord': 138,
# }

# station1 = 138 # Nord
# station2 = 139 # West
# station3 = 170 # Sud
# station4 = 171 # Ost

## All except SO2

In [ ]:
groups = {
    1: [138, 139, 170, ''], # Nord, West, Sud
    2: [171, 164, '', ''], # Ost, Don Bosco    
}
years = range(2009, 2021)

von_tag = 31
von_monat = 12
# von_jahr = 2020

bis_tag = 31
bis_monat = 12
# bis_jahr = 2021

mittelwert = 21 #mittelwert_vals['TMW (Tagesmittelwert)'] # 21: daily

cnt = 0
not_found = []

for von_jahr in years:
    for group_name, group in groups.items():
        for k, komp in komponente.items():                
            one_run = (k, von_jahr, group_name)
                
            print(f'Start: {one_run}')
            url = f'{link}station1={group[0]}&station2={group[1]}&komponente1={komp}&'\
                    f'station3={group[2]}&station4={group[3]}&komponente2={komp}&'\
                    f'von_tag={von_tag}&von_monat={von_monat}&von_jahr={von_jahr}&'\
                    f'mittelwert={mittelwert}&'\
                    f'bis_tag={bis_tag}&bis_monat={bis_monat}&bis_jahr={von_jahr+1}'
            try:
                driver = webdriver.Chrome(service=s)
                driver.get(url)
                time.sleep(2)
                a_tags = driver.find_elements(By.TAG_NAME, 'a')
                for a in a_tags:
                    if 'export.php' in a.get_attribute('href'):
                        a.click()
                        time.sleep(3)

                cnt += 1
                driver.quit()
            except: 
                print(f'\tnot found: {one_run}')
                not_found.append(one_run)
                driver.quit()

## Only SO2 for group 2: No data for Ost

In [ ]:
groups = {
    1: [138, 139, 170, ''], # Nord, West, Sud
    2: [164, '', '', ''], # Ost, Don Bosco    
}
# years = range(2010, 2021)
# years = [2009]

von_tag = 31
von_monat = 12
# von_jahr = 2020

bis_tag = 31
bis_monat = 12
# bis_jahr = 2021

mittelwert = 21 #mittelwert_vals['TMW (Tagesmittelwert)'] # 21: daily

cnt = 0
tracking = {}
not_found = []

for von_jahr in years:    
    for group_name, group in groups.items():
        if group_name != 2: 
            continue        
        for k, komp in komponente.items():
            if k != 'SO2': continue
            
            one_run = (k, von_jahr, group_name)                
            print(f'Start: {one_run}')
            url = f'{link}station1={group[0]}&station2={group[1]}&komponente1={komp}&'\
                    f'station3={group[2]}&station4={group[3]}&komponente2=&'\
                    f'von_tag={von_tag}&von_monat={von_monat}&von_jahr={von_jahr}&'\
                    f'mittelwert={mittelwert}&'\
                    f'bis_tag={bis_tag}&bis_monat={bis_monat}&bis_jahr={von_jahr+1}'
            try:
                driver = webdriver.Chrome(service=s)
                driver.get(url)
                time.sleep(2)
                a_tags = driver.find_elements(By.TAG_NAME, 'a')
                for a in a_tags:
                    if 'export.php' in a.get_attribute('href'):
                        a.click()
                        time.sleep(3)
                tracking[one_run] = cnt
                cnt += 1
                driver.quit()
            except: 
                print(f'\tnot found: {one_run}')
                not_found.append(one_run)
                driver.quit()


# Combine Files

In [ ]:
# get all column names
out = '../../output_intermediate/tmp'

all_params = set()
for file in os.listdir(out):
    print(file)      
    path = os.path.join(out, file)
    all_params = all_params.union(set(pd.ExcelFile(path).sheet_names))

cnt = 0

dfs = {}
df = pd.DataFrame(columns= all_params,
                 index=pd.date_range(start='2010-01-01', end='2010-12-31'))
for file in sorted(os.listdir(out)):
    print(file)
    path = os.path.join(out, file)
    sheet_names = pd.ExcelFile(path).sheet_names
    
    for sheet_name in sheet_names:
        tmp = pd.read_excel(path, header=4, sheet_name=sheet_name).drop('Uhrzeit', axis=1)
        tmp['Datum'] = pd.to_datetime(tmp['Datum'], dayfirst=True)
        tmp = tmp.set_index('Datum')
        tmp.rename(columns={'Wert': sheet_name}, inplace=True)
        df.loc[tmp.index, sheet_name] = tmp[sheet_name]

    cnt += 1
df = df[sorted(df.columns)]
df    

In [19]:
df1 = pd.read_csv('../../data/01_working_data/main_data.csv', index_col=0)
df1

,Graz-DB (LUFE),Graz-DB (LUTE),Graz-DB (NO),Graz-DB (NO2),Graz-DB (NOX),Graz-DB (PM10),Graz-DB (SO2),Graz-DB (STBK 10K),Graz-N (LUFE),Graz-N (LUTE),...,Graz-S (PM10),Graz-S (SO2),Graz-S (STBK 10K),Graz-W (LUFE),Graz-W (LUTE),Graz-W (NO),Graz-W (NO2),Graz-W (NOX),Graz-W (PM10),Graz-W (STBK 10K)
2011-01-01,96.85102,-5.908208,63.428520,58.980427,81.712555,99.820000,2.588490,117.761024,93.191414,-5.484686,...,84.840034,0.642840,103.788994,93.010414,-5.488395,46.887447,53.420410,65.541910,106.652306,106.652306
2011-01-02,95.49811,-6.005374,56.789352,51.368520,72.407650,136.100050,2.462043,193.199520,91.321380,-5.178950,...,121.799965,0.761799,168.451430,92.434390,-5.552729,51.401985,46.865738,65.734040,177.517240,177.517240
2011-01-03,96.50521,-4.467000,46.410530,54.802776,65.879585,82.250000,2.391510,107.475980,93.380640,-3.730794,...,75.389960,0.830818,94.543860,94.181670,-3.988200,34.175570,49.436160,53.263200,97.260414,97.260414
2011-01-04,88.10501,-4.897977,178.927440,88.957440,190.013170,70.990000,5.910404,77.401720,80.562570,-3.882670,...,69.699974,3.294453,78.027460,84.103940,-4.435886,88.947014,68.695320,107.244880,62.180237,62.180237
2011-01-05,88.32999,-6.524708,164.113740,73.731700,170.165740,72.790020,6.308275,77.671830,81.767420,-5.316336,...,63.580017,3.451762,68.269135,84.680830,-6.053708,86.238590,51.889156,96.293945,58.103577,58.103577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,97.38607,0.616328,9.010106,22.106157,18.798016,12.708243,1.201998,12.708243,92.065380,0.995206,...,12.715829,0.202835,12.715829,92.505684,1.511686,1.032345,14.319123,8.327889,10.261334,10.261334
2021-12-28,96.66538,0.309437,18.288683,32.319553,31.589369,18.057463,1.215151,18.057463,91.964550,0.749909,...,17.711733,0.143881,17.711733,91.901650,1.242483,1.997687,24.927174,14.651120,16.204187,16.204187
2021-12-29,97.59131,0.408755,21.886822,30.546669,33.549927,27.875114,1.412200,27.875114,93.031680,0.821417,...,24.525108,0.254251,24.525108,92.239390,1.374253,4.771584,25.525824,17.189781,27.156235,27.156235
2021-12-30,97.42017,2.230151,85.920690,39.675133,89.718290,24.512735,2.355048,24.512735,92.109150,2.887107,...,22.680038,1.519852,22.680038,90.843320,3.358200,41.332493,32.923622,50.400394,16.832644,16.832644


In [22]:
set(df1.columns) - set(df.columns)

{'Graz-O P (LUFE)', 'Graz-O P (LUTE)', 'Graz-O P (PM10)'}

In [38]:
df.index = [str(i.date()) for i in df.index]
df1 = pd.concat([df1, pd.DataFrame(index=df.index, columns=df1.columns)]).sort_index()
df1.loc[df.index, df.columns] = df.values
df1

,Graz-DB (LUFE),Graz-DB (LUTE),Graz-DB (NO),Graz-DB (NO2),Graz-DB (NOX),Graz-DB (PM10),Graz-DB (SO2),Graz-DB (STBK 10K),Graz-N (LUFE),Graz-N (LUTE),...,Graz-S (SO2),Graz-S (STBK 10K),Graz-W (LUFE),Graz-W (LUTE),Graz-W (NO),Graz-W (NO2),Graz-W (NOX),Graz-W (PM10),Graz-W (STBK 10K),Graz-W (SO2)
2010-01-01,83.532080,3.381270,115.450000,62.420425,125.220856,50.290020,7.902596,48.681210,77.251884,4.035931,...,4.347298,58.876480,83.031044,3.497470,69.225334,47.048294,80.099790,38.215210,38.215210,6.325872
2010-01-02,87.016050,2.954946,106.040840,49.886177,111.119576,116.870080,9.089468,148.307540,81.564156,3.522331,...,7.853193,169.943600,86.777100,3.033653,57.718700,35.495000,64.850220,79.188095,79.188095,4.752894
2010-01-03,55.582905,2.673754,40.915726,21.900616,44.270275,23.994997,3.700659,21.664154,56.786457,3.109104,...,1.448857,19.864191,56.488766,2.953737,14.706230,13.905152,19.065725,14.734432,14.734432,1.943021
2010-01-04,42.072296,-0.668366,36.093445,36.261917,47.911000,20.080004,4.100872,21.462568,49.110626,-0.742807,...,1.827445,26.680067,44.811253,-0.571709,7.937342,22.021637,17.883360,11.564521,11.564521,2.126345
2010-01-05,72.974790,-3.945791,99.885315,71.757225,117.629790,48.430010,6.698893,53.670230,69.331055,-3.867371,...,3.059063,54.818146,73.859795,-4.016794,29.179869,50.603188,49.865322,34.387577,34.387577,3.871213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,97.386070,0.616328,9.010106,22.106157,18.798016,12.708243,1.201998,12.708243,92.065380,0.995206,...,0.202835,12.715829,92.505684,1.511686,1.032345,14.319123,8.327889,10.261334,10.261334,NaN
2021-12-28,96.665380,0.309437,18.288683,32.319553,31.589369,18.057463,1.215151,18.057463,91.964550,0.749909,...,0.143881,17.711733,91.901650,1.242483,1.997687,24.927174,14.651120,16.204187,16.204187,NaN
2021-12-29,97.591310,0.408755,21.886822,30.546669,33.549927,27.875114,1.412200,27.875114,93.031680,0.821417,...,0.254251,24.525108,92.239390,1.374253,4.771584,25.525824,17.189781,27.156235,27.156235,NaN
2021-12-30,97.420170,2.230151,85.920690,39.675133,89.718290,24.512735,2.355048,24.512735,92.109150,2.887107,...,1.519852,22.680038,90.843320,3.358200,41.332493,32.923622,50.400394,16.832644,16.832644,NaN


In [42]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4383 entries, 2010-01-01 to 2021-12-31
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Graz-DB (LUFE)       4374 non-null   float64
 1   Graz-DB (LUTE)       4357 non-null   float64
 2   Graz-DB (NO)         4368 non-null   float64
 3   Graz-DB (NO2)        4368 non-null   float64
 4   Graz-DB (NOX)        4368 non-null   float64
 5   Graz-DB (PM10)       4327 non-null   float64
 6   Graz-DB (SO2)        4355 non-null   float64
 7   Graz-DB (STBK 10K)   4310 non-null   float64
 8   Graz-N (LUFE)        4365 non-null   float64
 9   Graz-N (LUTE)        4340 non-null   float64
 10  Graz-N (NO)          4368 non-null   float64
 11  Graz-N (NO2)         4368 non-null   float64
 12  Graz-N (NOX)         4368 non-null   float64
 13  Graz-N (PM10)        2683 non-null   float64
 14  Graz-N (SO2)         4355 non-null   float64
 15  Graz-N (STBK 10K)    4305 no

In [41]:
df1.to_csv('main_data.csv')

In [43]:
len(pd.date_range('2010-01-01', '2021-12-31'))

4383